In [58]:
import os
os.getcwd()
os.chdir("/mnt/workspace/Public-DSC291/notebooks/Section2-PCA/Full_Dataset_analysis")
os.getcwd()

from Startup import *

In [59]:
%%time
query="""
SELECT stations.ID, stations.elevation, stations.state, stations.latitude, stations.longitude,
        weather.Station, weather.Measurement, weather.Year, weather.Values
FROM stations
LEFT JOIN weather
ON stations.ID = weather.Station
WHERE stations.state <> '' AND weather.Measurement='PRCP'
"""

US_df = sqlContext.sql(query)
US_df.show(10)

+-----------+---------+-----+--------+---------+-----------+-----------+----+--------------------+
|         ID|elevation|state|latitude|longitude|    Station|Measurement|Year|              Values|
+-----------+---------+-----+--------+---------+-----------+-----------+----+--------------------+
|USC00042239|   1414.3|   CA| 32.9897|-116.5872|USC00042239|       PRCP|1899|[00 7E 00 7E 0C 5...|
|USC00042239|   1414.3|   CA| 32.9897|-116.5872|USC00042239|       PRCP|1900|[00 00 00 00 84 5...|
|USC00042239|   1414.3|   CA| 32.9897|-116.5872|USC00042239|       PRCP|1901|[00 49 00 00 00 0...|
|USC00042239|   1414.3|   CA| 32.9897|-116.5872|USC00042239|       PRCP|1902|[00 7E 00 7E 00 7...|
|USC00042239|   1414.3|   CA| 32.9897|-116.5872|USC00042239|       PRCP|1903|[00 00 00 00 00 0...|
|USC00042239|   1414.3|   CA| 32.9897|-116.5872|USC00042239|       PRCP|1904|[00 00 00 00 00 0...|
|USC00042239|   1414.3|   CA| 32.9897|-116.5872|USC00042239|       PRCP|1905|[7C 5C 00 00 00 0...|
|USC000422

In [61]:
%%time
import pyspark.sql.functions as F
import pyspark.sql.types as T

def getMean(row):
    new_row = np.nanmean(unpackArray(row,np.float16))
    return float(new_row)

my_udf = F.UserDefinedFunction(getMean, T.DoubleType())
US_df = US_df.withColumn('Values_avg', my_udf('Values'))
US_df.show(10)

+-----------+---------+-----+--------+---------+-----------+-----------+----+--------------------+----------+
|         ID|elevation|state|latitude|longitude|    Station|Measurement|Year|              Values|Values_avg|
+-----------+---------+-----+--------+---------+-----------+-----------+----+--------------------+----------+
|USC00042239|   1414.3|   CA| 32.9897|-116.5872|USC00042239|       PRCP|1899|[00 7E 00 7E 0C 5...|   128.125|
|USC00042239|   1414.3|   CA| 32.9897|-116.5872|USC00042239|       PRCP|1900|[00 00 00 00 84 5...| 21.140625|
|USC00042239|   1414.3|   CA| 32.9897|-116.5872|USC00042239|       PRCP|1901|[00 49 00 00 00 0...|   80.4375|
|USC00042239|   1414.3|   CA| 32.9897|-116.5872|USC00042239|       PRCP|1902|[00 7E 00 7E 00 7...| 25.421875|
|USC00042239|   1414.3|   CA| 32.9897|-116.5872|USC00042239|       PRCP|1903|[00 00 00 00 00 0...| 23.890625|
|USC00042239|   1414.3|   CA| 32.9897|-116.5872|USC00042239|       PRCP|1904|[00 00 00 00 00 0...| 27.984375|
|USC000422